<a href="https://colab.research.google.com/github/rojinadeuja/Multi-Layer-Perceptron/blob/master/NLP_Tasks_Using_Flair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Modules

In [0]:
# Install the PyDrive wrapper & import libraries.

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1GhyH4k9C4uPRnMAMKhJYOqa-V9Tqt4q8' ### File ID ###
data = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))

## Read the data

In [36]:
import io
import pandas as pd
data1 = drive.CreateFile({'id': file_id})
data1 = pd.read_csv(io.StringIO(data1.GetContentString())) 
data1.head()
print(data1.shape)
data1 = data1[data1['label'].notna()]
print(data1.shape)

(49159, 3)
(31962, 3)


## Load Flair and PyTorch Library

In [0]:
# Remove all rows that have label as NaN
data = data[data['label'].notna()]
# Sample only a 1000 rows
data= data.sample(1000)

In [19]:
# Check data dimensions
data.shape

(1000, 3)

In [20]:
import torch
# !pip install flair
!pip install --upgrade git+https://github.com/flairNLP/flair.git
import flair

  Cloning https://github.com/flairNLP/flair.git to /tmp/pip-req-build-0ezsm5fk
  Running command git clone -q https://github.com/flairNLP/flair.git /tmp/pip-req-build-0ezsm5fk
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 256kB 2.8MB/s 
     |████████████████████████████████| 798kB 48.6MB/s 
     |████████████████████████████████| 983kB 37.7MB/s 
     |████████████████████████████████| 645kB 40.5MB/s 
     |████████████████████████████████| 1.1MB 39.4MB/s 
     |████████████████████████████████| 890kB 47.3MB/s 
     |████████████████████████████████| 3.8MB 38.0MB/s 
  Created wheel for flair: filename=flair-0.4.5-cp36-none-any.whl size=148505 sha256=b7f11378af7bcd11b7e1ffb9e217dbbbfbd0dd521385f1db8c63ae2a9e9bcd36
  Stored in directory: /tmp/pip-ephem-wheel-cache-gi263y5r/wheels/84/82/73/d2b3b59b7be74ea05f2c6d64132efe27df52daffb47d1dc7bb
Successfully built flair
  Crea

In [21]:
from flair.data import Sentence
# Create a sentence
sentence = Sentence('Blogs of Analytics Vidhya are Awesome.')
# Print the sentence to see what’s in it
print(sentence) # A Sentence is essentially a list of tokens

Sentence: "Blogs of Analytics Vidhya are Awesome."   [− Tokens: 6]


In [22]:
# Extract only the tweet column from the dataframe
text = data['tweet'] 
# Create a list fo the tweets called txt
txt = text.tolist()
print(txt[:10])

['  user really   that  user are sponsoring  user this year  thegamefair  ', ' user irresponsible parents creating irresponsible kids who don t care abt laws     ', '  user he make me happy     myangel      me  selfie  blonde  picoftheday  beard  beards  boy    ', 'i am thankful for love   thankful  positive     ', ' user you make me and all the  user proud  we feel better when you   makeamericahate     thank a rube today  alwaystrump', 'tonight s the night     mtg  eternalmasters  boosterdraft     joshua tree games ', 'use the power of your mind to  heal your body      altwaystoheal  healthy is     ', 'work in progress for  summer       newstyles  inspiration  finebyme  photooftheday    ', 'couldn t be happier with her         her  loveislove  love  gay      ', ' coffeelover   attack bull game  d  do you really think that his head was empty around the city  each side i ']


## Word Embedding Using Flair

In [23]:
# Import the embeddings
from flair.embeddings import WordEmbeddings
from flair.embeddings import CharacterEmbeddings
from flair.embeddings import StackedEmbeddings
from flair.embeddings import FlairEmbeddings
from flair.embeddings import BertEmbeddings
from flair.embeddings import ELMoEmbeddings
from flair.embeddings import FlairEmbeddings

# Initialise embeddings (un-comment to use others)
#glove_embedding = WordEmbeddings('glove')
#character_embeddings = CharacterEmbeddings()
flair_forward  = FlairEmbeddings('news-forward-fast')
flair_backward = FlairEmbeddings('news-backward-fast')
#bert_embedding = BertEmbedding()
#elmo_embedding = ElmoEmbedding()

# Stack the embeddings : Combine multiple embeddings into a powerful word representation model without much complexity
stacked_embeddings = StackedEmbeddings( embeddings = [ 
                                                       flair_forward, 
                                                       flair_backward
                                                      ])

2020-05-21 16:03:21,792 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-forward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmpm__6bk_b


100%|██████████| 19689779/19689779 [00:01<00:00, 12939282.50B/s]

2020-05-21 16:03:23,847 copying /tmp/tmpm__6bk_b to cache at /root/.flair/embeddings/lm-news-english-forward-1024-v0.2rc.pt
2020-05-21 16:03:23,876 removing temp file /tmp/tmpm__6bk_b


2020-05-21 16:03:24,885 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-backward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmp_q76m2fe


100%|██████████| 19689779/19689779 [00:01<00:00, 13045611.71B/s]

2020-05-21 16:03:26,910 copying /tmp/tmp_q76m2fe to cache at /root/.flair/embeddings/lm-news-english-backward-1024-v0.2rc.pt
2020-05-21 16:03:26,938 removing temp file /tmp/tmp_q76m2fe


## Test stacked embeddings



In [24]:
# Create a sentence
sentence = Sentence('These blogs are awesome.')
# Embed words in the sentence
stacked_embeddings.embed(sentence)
for token in sentence:
  print(token.embedding)
# Print type and size of the embedding
print(type(token.embedding))
print(token.embedding.size()[0])
z = token.embedding.size()[0]

tensor([ 3.8988e-03, -3.7598e-05, -3.4552e-03,  ...,  1.1473e-09,
        -7.2948e-07,  5.2823e-02])
tensor([-4.9989e-02, -6.4013e-05,  6.3951e-03,  ...,  3.6644e-09,
         4.5787e-07, -6.5868e-02])
tensor([-4.9550e-03, -1.2693e-04,  2.1048e-02,  ...,  1.2971e-07,
         4.0299e-07, -1.2174e-01])
tensor([-7.3077e-05, -5.1575e-06,  2.3907e-02,  ...,  2.4966e-08,
         6.5300e-09, -1.1079e-01])
<class 'torch.Tensor'>
2048


## Vectorizing the text
We will be using two approaches for vectorizing the text.

## Mean of Word Embeddings within a Tweet
In this approach, for each sentence we do the following:
1. Generate word embeddings for each word
2. Calculate mean of the embeddings for each word to get embedding of the sentence

In [25]:
from tqdm import tqdm # Progress bar to track progress
# Create a tensor for storing the sentence embeddings
s = torch.zeros(0, z)

# Iterate over each sentence
for tweet in tqdm(txt):
  # Empty tensor for words
  w = torch.zeros(0, z)
  sentence = Sentence(tweet)
  stacked_embeddings.embed(sentence)
  # Iterate over each word
  for token in sentence:
    # Store the embeddings of each word in a sentence
    w = torch.cat((w, token.embedding.view(-1, z)), 0)
    # Store the embeddings of each sentence ie. all words
    s = torch.cat((s, w.mean(dim = 0).view(-1, z)), 0)

100%|██████████| 1000/1000 [05:18<00:00,  3.14it/s]


## Document Embedding: Vectorizing the entire Tweet

In [0]:
# from flair.embeddings import DocumentPoolEmbeddings

# # Initialize the document embeddings, mode =mean
# document_embeddings = DocumentPoolEmbeddings([
#                                               flair_backward,
#                                               flair_forward
# ])

# # Store the size of embedding
# z = sentence.embedding.size()[0]

# # Vectorize the text
# # Create a tensor for storing sentence embeddings
# s = torch.zeros(0, z)

# # Iterate over the sentences
# for tweet in tqdm(txt):
#   sentence = Sentence(tweet)
#   document_embeddings.embed(sentence)
#   # Adding Document embeddings to list
#   s = torch.cat((s, sentence.embedding.view(-1,z)), 0)


## Partition the data into Train/Test sets

In [0]:
X = s.numpy() # Convert the tensor to a numpy array

# Test Set
test = X[1000:, :]
train = X[:1000, :]

In [38]:
# Check the dimensions for the train dataset
train.shape

(1000, 2048)

In [39]:
# Check the dimensions for the train dataset
test.shape

(12022, 2048)

In [40]:
# Extract labels of the training set
target = data['label'][data['label'].isnull()==False].values
target.shape

(1000,)

## Build the Model and Define Custome Evaluator (for F1 Score)

In [0]:
# Define custom F1 evaluator for XGBoost
def custom_eval(preds, dtrain):
  labels = dtrain.get_label().astype(np.int)
  preds = (preds >= 0.3).astype(np.int)
  return[('f1_score', f1_score(labels, preds))]

In [45]:
# Build the XGBoost model
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Split the train and test set
x_train, x_valid, y_train, y_valid = train_test_split(train, target, random_state=42, test_size=0.3)

# XGBoost compatible data
dtrain = xgb.DMatrix(x_train, y_train)
dvalid = xgb.DMatrix(x_valid, label = y_valid)

# Define the parameters
params = {
    'colsample': 0.9,
    'colsample_bytree': 0.5,
    'eta': 0.1,
    'max_depth': 8,
    'min_child_weight': 6,
    'objective': 'binary:logistic',
    'subsample': 0.9
}

# Train the model
xgb_model = xgb.train(params, dtrain, feval=custom_eval, num_boost_round=1000, maximize=True, 
                      evals=[(dvalid, "Validation")], early_stopping_rounds=100)

[0]	Validation-error:0.053333	Validation-f1_score:0.101266
Multiple eval metrics have been passed: 'Validation-f1_score' will be used for early stopping.

Will train until Validation-f1_score hasn't improved in 100 rounds.
[1]	Validation-error:0.053333	Validation-f1_score:0.101266
[2]	Validation-error:0.053333	Validation-f1_score:0.101266
[3]	Validation-error:0.053333	Validation-f1_score:0.101266
[4]	Validation-error:0.053333	Validation-f1_score:0.101266
[5]	Validation-error:0.053333	Validation-f1_score:0.075472
[6]	Validation-error:0.053333	Validation-f1_score:0.060606
[7]	Validation-error:0.053333	Validation-f1_score:0.061538
[8]	Validation-error:0.053333	Validation-f1_score:0
[9]	Validation-error:0.053333	Validation-f1_score:0
[10]	Validation-error:0.053333	Validation-f1_score:0
[11]	Validation-error:0.053333	Validation-f1_score:0
[12]	Validation-error:0.053333	Validation-f1_score:0
[13]	Validation-error:0.053333	Validation-f1_score:0
[14]	Validation-error:0.053333	Validation-f1_sco

## Make Predictions


In [0]:
# Reformatting test set for XGB
dtest = xgb.DMatrix(test)

# Prediction
predict = xgb_model.predict(dtest) # predicting